In [ ]:
from datetime import datetime
from datetime import timedelta

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk 
import re
%matplotlib inline

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [ ]:
#from ekphrasis.classes.segmenter import Segmenter
from wordsegment import load, segment
load()

In [ ]:
df = pd.read_csv('twitter.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'],format="%Y-%m-%d %H:%M:%S")
df =df.set_index('timestamp')

In [ ]:
df9 = pd.DataFrame()

In [ ]:
df9 = df[df.index.month == 9]

In [ ]:
df9 = df9.copy()

In [ ]:
def extract_hashtags(s):
    s = re.findall(r"#(\w+)", s)
    return ' '.join(s)
df9['all_hashtags'] = df9['tweets'].apply(lambda x: extract_hashtags(x))

In [ ]:
def segment_hashtags(s):
    s = ' '.join(segment(s))
    #hashtag_tokens=word_tokenize(string)
    #return " ".join(hashtag_tokens)
    return s
df9['all_hashtags_seg'] = df9['all_hashtags'].apply(lambda x: segment_hashtags(x))

In [ ]:
punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”…’'

def reduce_noise(text):
    ##mentions, hashtags, urls and emojis
    text = re.sub(r"#(\w+)",'', text)
    text = re.sub(r"@\w+",'', text)
    text = re.sub(r'\\x..','',text)
    text = re.sub("\n",'',text)
    text = re.sub(r"((http|https|ftp|file|www)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|])",'', text)
    ##html entities
    text = re.sub(r"&amp;", "&", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r'[0-9]+', '', text)#'\d+'
    text = "".join([i for i in text if i not in punc])
    return text
df9['tweets_clean'] = df9['tweets'].apply(lambda x: reduce_noise(x))

In [ ]:
sw = stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in sw]
    return text

In [ ]:
df9['tweets_token'] = df9['tweets_clean'].apply(lambda x: word_tokenize(x.lower()))
df9['tweets_nonstop'] = df9['tweets_token'].apply(lambda x: remove_stopwords(x))

df9['tags_token'] = df9['all_hashtags_seg'].apply(lambda x: word_tokenize(x.lower()))
df9['tags_nonstop'] = df9['tags_token'].apply(lambda x: remove_stopwords(x))

In [ ]:
from collections import Counter

corpus = [item for sublist in df9['tags_nonstop'] for item in sublist]
counter = Counter(corpus)

df_count = pd.DataFrame(counter.most_common(30))
df_count.columns = ['Common_words','count']
df_count.style.background_gradient(cmap = 'Blues')

In [ ]:
arm_aze_label = ['azerbaijan','armenia','armenian','azerbaijani','karabakh','nagorno','artsakh','aliyev']
usa_label = ['trump','obama','usa','harris','biden']
turkey_label = ['erdogan','turkey','istanbul']
covid_label = ['covid','covid19','corona','virus','pandemic','coronavirus']
yemen_label = ['yemen','yemeni']
africa_label = ['africa','ethiopia']

In [ ]:
def get_labels(text): 
    if any(w in text for w in arm_aze_label):
        return 'Azerbaijan_Armenia'
    elif any(w in text for w in turkey_label): 
        return 'Turkey'
    elif any(w in text for w in usa_label): 
        return 'USA'
    elif any(w in text for w in covid_label): 
        return 'covid'
    elif any(w in text for w in yemen_label): 
        return 'Yemen'
    elif any(w in text for w in africa_label): 
        return 'Africa'
    else: 
        return 'Other'
    
df9['label'] = df9['all_hashtags_seg'].apply(lambda x: get_labels(x))

In [ ]:
df9 = df9[~df9['label'].isin(['Other'])]

In [ ]:
#ps = nltk.PorterStemmer()
#def stemming(text):
    #text = [ps.stem(word) for word in text]
    #return text

#wl = nltk.WordNetLemmatizer()
#def lemmatizer(text):
    #text = [wl.lemmatize(word) for word in text]
    #return text

In [ ]:
#df['tweets_stem'] = df['tweets_nonstop'].apply(lambda x: stemming(x))
#df['tags_stem'] = df['tags_nonstop'].apply(lambda x: stemming(x))
#df['tweets_lem'] = df['tweets_stem'].apply(lambda x: lemmatizer(x))
#df['tags_lem'] = df['tags_stem'].apply(lambda x: lemmatizer(x))
#df['tweets_pos'] = df['tweets_stem'].apply(lambda x: nltk.pos_tag(x))
#df['tags_pos'] = df['tags_stem'].apply(lambda x: nltk.pos_tag(x))

In [ ]:
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
#locator = Nominatim(user_agent='myGeocoder')
#geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
#df['geo_location'] = df['location'].apply(geocode)
#df['point'] = df['geo_location'].apply(lambda loc: tuple(loc.point) if loc else None)
#df[['latitude', 'longitude','altitude']] = pd.DataFrame(df['point'].tolist(), index=clean_tweets.index)